In [ ]:
import matplotlib.pyplot as plt
import src.geneticAlgorithm.algorithmTestUtils as tst
from src.geneticAlgorithm.geneticAlgorithm import GeneticAlgorithm
from src.witnessproblem import Instance, RandomInstanceGenerator

NUM_INSTANCES = 5
RUNS_PER_INSTANCE = 10

In [ ]:
import src.witnessproblem.randomGenerator as generator

NUM_VERTICES = [50, 100, 150, 200, 250]
for i, numVertices in enumerate(NUM_VERTICES):
    RandomInstanceGenerator(vertices=numVertices, 
                            avg_vertex_degree=5, 
                            max_time_window=60, 
                            max_vertex_distance=5, 
                            witnesses=500, 
                            max_interval_length=10,
                            max_vertices_per_testimony=3,
                            negative_testimonies_rate=0.2).create(f"tuning/I{i}.txt", 1)


In [ ]:
def executeOnInstance(algorithms: GeneticAlgorithm, filename, repetitions=1):

    print(f"Running instance {filename}")
    with open('instances/' + filename + '.txt') as inFile:

        num_instances = int(inFile.readline())
        meanSolutions = []
        maxSolutions = []

        if num_instances != 1:
            raise "ExecuteOnInstance only supports files with once instance for now!"

        instance = Instance()
        instance.readFromFile(inFile) 

        for alg in algorithms:
            solution = list()
            for _ in range(repetitions):
                solution.append(alg.run(instance))

            meanSolutions.append(sum(solution)/repetitions)
            maxSolutions.append(max(solution))

    return meanSolutions, maxSolutions

In [ ]:
def runAndGenerateCoolGraphs(algorithms, testedParameters, parameterName):
    for i in range(NUM_INSTANCES):
        meanSolutions, maxSolutions = executeOnInstance(algorithms, f'tuning/I{i}', repetitions=RUNS_PER_INSTANCE)
        print(f"\nInstance #{i}: \nMean solutions per algorithm: {meanSolutions}\tMax solutions per algorithm: {maxSolutions}")

        plt.plot(testedParameters, meanSolutions, label="mean fitness")
        plt.plot(testedParameters, maxSolutions, label="max fitness")
        plt.legend()
        plt.xlabel(parameterName)
        plt.savefig(f"solutions/tuning/{parameterName}I{i}")
        plt.clf()

        tst.executeAlgorithmsGraph(algorithms, f'tuning/I{i}', legend=testedParameters)
        plt.title(parameterName)
        plt.savefig(f"solutions/tuning/comparison/{parameterName}I{i}")
        plt.clf()

#### Population Size Tuning

As expected, the results show us that using populations of bigger sizes improves the results obtained by the
algorithm. However, this performance boost seems to reach a limit between populations of size 150 and 200, being
hardly any difference between the two values. Therefore, making a trade-off between computational time and efficiency,
we will use a population of 150 individuals for the subsequent experiments.

In [ ]:
populationSizes = [50, 75, 100, 150, 200]
algorithms = [GeneticAlgorithm(
    numGenerations = 100, 
    populationSize = size,
    localSearchProb=lambda x, y: 0
    ) for size in populationSizes]

runAndGenerateCoolGraphs(algorithms, populationSizes, "Population Size")

SELECTED_POPULATION = 150

#### Mutation Probability Tuning

We can also see that there is no significant difference between the
values obtained when the mutation probability remains in the ranges 𝑝𝑚 ∈ [0.3, 0.9].
For obvious reasons, values close to 0 lead to the worst results, but the experiments do not show much sensitivity to the
parameter when it is within this range. The value we will use for the forthcoming experiments is 𝑝𝑚 = 0.6

In [ ]:
mutationProbs = [0,0.2, 0.4, 0.6, 0.8, 1]
algorithms = [GeneticAlgorithm(
    numGenerations = 100, 
    populationSize = SELECTED_POPULATION, 
    mutationProbability=prob,
    localSearchProb=lambda x, y: 0
    ) for prob in mutationProbs]

runAndGenerateCoolGraphs(algorithms, mutationProbs, "Mutation Probability")

SELECTED_MUTATION = 0.6

#### Crossover Probability Tuning

Similarly, there is no significant difference in the outcomes when the crossover probability
remains in the ranges 𝑝𝑐 ∈ [0.5, 0.9].
The value we will use for the forthcoming experiments is 𝑝𝑐 = 0.7

In [ ]:
crossoverProbs = [0.1, 0.3, 0.5, 0.7, 0.9]
algorithms = [GeneticAlgorithm(
    numGenerations = 100, 
    populationSize = SELECTED_POPULATION, 
    mutationProb = SELECTED_MUTATION
    crossoverRate=prob, 
    localSearchProb=lambda x, y: 0
    ) for prob in crossoverProbs]

runAndGenerateCoolGraphs(algorithms, crossoverProbs, "Crossover Proportion")

SELECTED_CROSSOVER = 0.7

### Local Search Probability Tuning

When it comes to local search probability, we will perform three experiments:
- A fixed probability
- A increasingly linear probability, that starts with an initial value of 0, and increase linearly in the number
of iterations of the algorithm, until it reaches a maximum value of 𝑝𝑙𝑠 (in particular, if 𝑝𝑙𝑠 = 0 then the memetic algorithm behaves
exactly as a genetic algorithm for the whole execution). 
- A probability that increases quadraticly until it reaches the maximum.

The local search operator considerably increases the execution time of the algorithm, so we need to find a balance between effectiveness and performance. 
Results suggest that the lineal approach, caped at 0.3, is an appropriate value, but the graphs are great examples of how higher values of this probability can
make the algorithm fall too early in local maxima. Considering a trade-off between time, the results obtained, and the
possibility of premature convergence, we will choose 𝑝𝑙𝑠 = 0.2.

In [ ]:
maxLocalSearchProbs = [0, 0.1, 0.2, 0.3, 0.4]
localSearchProbs = [lambda _,x : maxProb for maxProb in maxLocalSearchProbs]
algorithms = [GeneticAlgorithm(
    numGenerations = 100, 
    populationSize = SELECTED_POPULATION, 
    mutationProb = SELECTED_MUTATION,
    crossoverRate = SELECTED_CROSSOVER,
    localSearchProb = prob
    ) for prob in localSearchProbs]

runAndGenerateCoolGraphs(algorithms, maxLocalSearchProbs, "Local Search Probability")

In [ ]:
maxLocalSearchProbs = [0, 0.1, 0.2, 0.3, 0.4]
localSearchProbs = [lambda it,maxIt: it/maxIt * maxProb for maxProb in maxLocalSearchProbs]
algorithms = [GeneticAlgorithm(
    numGenerations = 100, 
    populationSize = SELECTED_POPULATION, 
    mutationProb = SELECTED_MUTATION,
    crossoverRate = SELECTED_CROSSOVER,
    localSearchProb = prob) for prob in localSearchProbs]

runAndGenerateCoolGraphs(algorithms, maxLocalSearchProbs, "Max Local Search Probability")

In [ ]:
maxLocalSearchProbs = [0.1, 0.2, 0.3]
localSearchProbs = [lambda it,maxIt: (it/maxIt)**2 * maxProb for maxProb in maxLocalSearchProbs]
algorithms = [GeneticAlgorithm(
    numGenerations = 100, 
    populationSize = SELECTED_POPULATION, 
    mutationProb = SELECTED_MUTATION,
    crossoverRate = SELECTED_CROSSOVER,
    localSearchProb = prob) for prob in localSearchProbs]

runAndGenerateCoolGraphs(algorithms, maxLocalSearchProbs, "MaxLocalSearchSquaredProbability")

In [ ]:
localSearchProbs = [
    lambda it,maxIt: 0.1,
    lambda it,maxIt: 0.1 * it/maxIt,
    lambda it,maxIt: 0.1 *(it/maxIt)**2 
    ]
algorithms = [GeneticAlgorithm(
    numGenerations = 100, 
    populationSize = SELECTED_POPULATION, 
    mutationProb = SELECTED_MUTATION,
    crossoverRate = SELECTED_CROSSOVER,
    localSearchProb = prob) for prob in localSearchProbs]

runAndGenerateCoolGraphs(algorithms, ['constant 0.2', 'lineal 0.3', 'squared 0.4'], "LocalSearchFunctions")